In [1]:
import torch
from PIL import Image
from torchvision import transforms, models
from sys import version
version

'3.9.5 (default, Aug 29 2021, 19:01:31) \n[GCC 9.3.0]'

In [18]:
import torch
from collections import OrderedDict, Counter
from functools import partial
from dataclasses import dataclass
from typing import Tuple, List
from copy import deepcopy
from math import prod
from sys import version
version

@dataclass
class LayerDimensions:
    kernel_size: Tuple[int, int]
    stride: Tuple[int, int]
    padding: Tuple[int, int]
    input_size: List[int]
    output_size: List[int]


class ModelStatCollector:
    def __init__(self):
        self.model_stats = OrderedDict()
        self.hooks = []
        
    def __get_next_conv_layers(self, model):
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.modules.conv.Conv2d):
                yield (name, module)

    def __extract_stats(self, name, module, input, output):
        self.model_stats[name] = LayerDimensions(module.kernel_size, module.stride, module.padding, input_size=list(
            input[0].size()), output_size=list(output[0].size()))

    def __attach_collection_hooks_to_model(self, model):

        for name, conv_layer in self.__get_next_conv_layers(model):
            layer_collector = partial(self.__extract_stats, name)
            self.hooks.append(
                conv_layer.register_forward_hook(layer_collector))

    def __detach_stats_collection_hooks(self):
        for hook in self.hooks:
            hook.remove()

    def __reset(self):
        self.model_stats = {}
        self.hooks = []

    def collect_stats_from_model(self, model, input_batch):
        self.__attach_collection_hooks_to_model(model)
        model.eval()
        with torch.no_grad():
            model(input_batch)
        self.__detach_stats_collection_hooks()
        collected_stats = deepcopy(self.model_stats)
        self.__reset()
        return collected_stats

class ModelStatAnalyser:
    @ classmethod
    def get_kernel_stats(cls, model_stats):
        kernel_size_counter = Counter()
        stride_counter_dict = {}
        for layer in model_stats.values():
            kernel_size = layer.kernel_size
            stride = layer.stride
            kernel_size_counter.update(str(kernel_size[0]))
            if str(kernel_size[0]) not in stride_counter_dict:
                stride_counter_dict[str(kernel_size[0])] = Counter()
            stride_counter_dict[str(kernel_size[0])].update(str(stride[0]))
        return kernel_size_counter

    @ classmethod
    def get_intermediate_layer_sizes(cls, model_stats):
        intermediate_layer_sizes = [
            prod(layer.input_size) for layer in model_stats.values()]
        intermediate_layer_sizes.append(
            prod(list(model_stats.values())[-1].output_size))
        return intermediate_layer_sizes

    @ classmethod
    def get_intermediate_layer_size_bounds(cls, model_stats):
        intermediate_layer_sizes = cls.get_intermediate_layer_sizes(model_stats)
        return (max(intermediate_layer_sizes),
                min(intermediate_layer_sizes))

    @ classmethod
    def get_ub_input_size(cls, model_stats):
        return max([prod(layer.kernel_size[0]) for layer in model_stats.values()])

    @ classmethod
    def get_in_channel_stats(cls, model_stats):
        in_channel_dict = {}
        for layer in model_stats.values():
            kernel_size = layer.kernel_size
            if str(kernel_size[0]) not in in_channel_dict:
                in_channel_dict[str(kernel_size[0])] = {}
            if str(layer.input_size[1]) not in in_channel_dict[str(kernel_size[0])]:
                in_channel_dict[str(kernel_size[0])][str(layer.input_size[1])] = 0
            in_channel_dict[str(kernel_size[0])][str(layer.input_size[1])] += 1

        return in_channel_dict

    @ classmethod
    def get_filter_stats(cls, model_stats):
        out_channel_dict = {}
        for layer in model_stats.values():
            kernel_size = layer.kernel_size
            if str(kernel_size[0]) not in out_channel_dict:
                out_channel_dict[str(kernel_size[0])] = {}
            if str(layer.output_size[0]) not in out_channel_dict[str(kernel_size[0])]:
                out_channel_dict[str(kernel_size[0])][str(layer.output_size[0])] = 0
            out_channel_dict[str(kernel_size[0])][str(layer.output_size[0])] += 1

        return out_channel_dict

    @ classmethod
    def get_stride_stats(cls, model_stats):
        stride_counter_dict = {}
        for layer in model_stats.values():
            kernel_size = layer.kernel_size
            stride = layer.stride
            if str(kernel_size[0]) not in stride_counter_dict:
                stride_counter_dict[str(kernel_size[0])] = Counter()
            stride_counter_dict[str(kernel_size[0])].update(str(stride[0]))
        return stride_counter_dict

    @ classmethod 
    def get_models_stats_dict(cls, model_dict, input_batch, ssd_input_batch = None):
        collector = ModelStatCollector()
        stats_dict = {}
        raw_stats_dict = {}
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            if ssd_input_batch is not None:
                ssd_input_batch = ssd_input_batch.to('cuda')
        for model_name, model in model_dict.items():
            print('Analysing {}'.format(model_name))
            model.to('cuda')
            if model_name == 'ssd' and ssd_input_batch is not None:
                model_stats = collector.collect_stats_from_model(model, ssd_input_batch)
            else:
                model_stats = collector.collect_stats_from_model(model, input_batch)
                
            model.to('cpu')
            raw_stats_dict[model_name] = model_stats
            stats_dict[model_name] = {'kernel': ModelStatAnalyser.get_kernel_stats(model_stats),
                                    'stride': ModelStatAnalyser.get_stride_stats(model_stats),
                                    'in_channel': ModelStatAnalyser.get_in_channel_stats(model_stats),
                                    'filters': ModelStatAnalyser.get_filter_stats(model_stats),
                                    'intermediate_layer_bounds': ModelStatAnalyser.get_intermediate_layer_size_bounds(model_stats)}
        return stats_dict, raw_stats_dict
    
class ModelStatsAggregator:
    
    @ classmethod
    def get_aggregate_kernel_stats_as_percentages(cls, stats_dict):
        aggregate_kernel_stats = Counter()
        for model, stats in stats_dict.items():
            aggregate_kernel_stats += stats['kernel']
        ksize, counts = zip(*[(ksize, count) for ksize, count in aggregate_kernel_stats.items()])
        total_kernels = sum(counts)
        aggregate_kernel_stats_percentages = {ksize: counts/total_kernels for ksize, counts in zip(ksize,counts)}
        return aggregate_kernel_stats_percentages

    @ classmethod
    def get_aggregate_stride_stats_per_kernel(cls, stats_dict):
        aggregate_stride_stats = {}
        for model, stats in stats_dict.items():
            for kernel, counter in stats['stride'].items():
                if kernel not in aggregate_stride_stats:
                    aggregate_stride_stats[kernel] = Counter()
                aggregate_stride_stats[kernel] += counter
        return aggregate_stride_stats
    
    @ classmethod
    def get_stride_stats_per_kernel_as_percentages(cls, stats_dict):
        aggregate_stride_stats = cls.get_aggregate_stride_stats_per_kernel(stats_dict)
        aggregate_stride_stats_percentages = {}
        for ksize, stride_counter in aggregate_stride_stats.items():
            total_kernels = sum(stride_counter.values())
            aggregate_stride_stats_percentages[ksize] = {stride: count/total_kernels for stride, count in dict(stride_counter).items()}
        return aggregate_stride_stats_percentages
    
    @ classmethod
    def get_aggregate_in_channel_stats_per_kernel(cls, stats_dict):
        aggregate_in_channel_stats = {}
        for _, stats in stats_dict.items():
            for kernel, channel_dict in stats['in_channel'].items():
                if kernel not in aggregate_in_channel_stats:
                    aggregate_in_channel_stats[kernel] = {}
                for channel, count in channel_dict.items():
                    if channel not in aggregate_in_channel_stats[kernel]:
                        aggregate_in_channel_stats[kernel][channel] = 0
                    aggregate_in_channel_stats[kernel][channel] += count
        for kernel, channel_dict in aggregate_in_channel_stats.items():
            channel_dict = {k: v for k, v in sorted(channel_dict.items(), key=lambda item: item[1], reverse=True)}
            aggregate_in_channel_stats[kernel] = channel_dict
        return aggregate_in_channel_stats
    
    @ classmethod
    def get_aggregate_in_channel_stats(cls, stats_dict):
        aggregate_in_channel_stats = cls.get_aggregate_in_channel_stats_per_kernel(stats_dict)
        cross_kernel_aggregate_channel = {}
        for channel_dicts in aggregate_in_channel_stats.values():
            for channel_size, count in channel_dicts.items():
                if channel_size not in cross_kernel_aggregate_channel:
                    cross_kernel_aggregate_channel[channel_size] = 0
                cross_kernel_aggregate_channel[channel_size] += count
        cross_kernel_aggregate_channel = {k: v for k,v in sorted(cross_kernel_aggregate_channel.items(), key=lambda item: item[1], reverse=True)}
        return cross_kernel_aggregate_channel
        
    @ classmethod
    def get_aggregate_filter_stats_per_kernel(cls, stats_dict):
        aggregate_filter_stats = {}
        for model, stats in stats_dict.items():
            for kernel, channel_dict in stats['filters'].items():
                if kernel not in aggregate_filter_stats:
                    aggregate_filter_stats[kernel] = {}
                for channel, count in channel_dict.items():
                    if channel not in aggregate_filter_stats[kernel]:
                        aggregate_filter_stats[kernel][channel] = 0
                    aggregate_filter_stats[kernel][channel] += count
        for kernel, channel_dict in aggregate_filter_stats.items():
            channel_dict = {k: v for k, v in sorted(channel_dict.items(), key=lambda item: item[1], reverse=True)}
            aggregate_filter_stats[kernel] = channel_dict
        return aggregate_filter_stats

In [3]:
model_dict = {
    'ssd': torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd'),
    'lenet': torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True),
    'yolov5s': torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True),
    'yolov5m': torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True),
    'yolov5l': torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True),
    'yolov5x': torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True),
    'alexnet': models.alexnet(pretrained=True, progress=True),
    'vgg_11': models.vgg11(pretrained=True, progress=True),
    'vgg_13': models.vgg13(pretrained=True, progress=True),
    'vgg_16': models.vgg16(pretrained=True, progress=True),
    'vgg_19': models.vgg19(pretrained=True, progress=True),
    'vgg_11_bn': models.vgg11_bn(pretrained=True, progress=True),
    'vgg_13_bn': models.vgg13_bn(pretrained=True, progress=True),
    'vgg_16_bn': models.vgg16_bn(pretrained=True, progress=True),
    'vgg_19_bn': models.vgg19_bn(pretrained=True, progress=True),
    'resnet_18': models.resnet18(pretrained=True, progress=True),
    'resnet_34': models.resnet34(pretrained=True, progress=True),
    'resnet_50': models.resnet50(pretrained=True, progress=True),
    'resnet_101': models.resnet101(pretrained=True, progress=True),
    'resnet_152': models.resnet152(pretrained=True, progress=True),
    'squeezenet_1_0': models.squeezenet1_1(pretrained=True, progress=True),
    'squeezenet_1_1': models.squeezenet1_0(pretrained=True, progress=True),
    'densenet_121': models.densenet121(pretrained=True, progress=True),
    'densenet_169': models.densenet169(pretrained=True, progress=True),
    'densenet_201': models.densenet201(pretrained=True, progress=True),
    'densenet_161': models.densenet161(pretrained=True, progress=True),
    'inception_v3': models.inception_v3(pretrained=True, progress=True),
    'googlenet': models.googlenet(pretrained=True, progress=True),
    'shufflenet_v2_x0_5': models.shufflenet_v2_x0_5(pretrained=True, progress=True),
    'shufflenet_v2_x1_0': models.shufflenet_v2_x1_0(pretrained=True, progress=True),
    'mobilenet_v2': models.mobilenet_v2(pretrained=True, progress=True),
    'mobilenet_v3_large': models.mobilenet_v3_large(pretrained=True, progress=True),
    'mobilenet_v3_small': models.mobilenet_v3_small(pretrained=True, progress=True),
    'resnext_50_32x4d': models.resnext50_32x4d(pretrained=True, progress=True),
    'resnext_101_32x8d': models.resnext101_32x8d(pretrained=True, progress=True),
    'wide_resnet_50_2': models.wide_resnet50_2(pretrained=True, progress=True),
    'wide_resnet_101_2': models.wide_resnet101_2(pretrained=True, progress=True),
    'mnasnet0_5': models.mnasnet0_5(pretrained=True, progress=True),
    'mnasnet1_0': models.mnasnet1_0(pretrained=True, progress=True),
}

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /home/sultan/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 308 layers, 21356877 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 392 layers, 47025981 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics

In [4]:
# prepare sample inputs
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg'
]
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

inputs = [utils.prepare_input(uri) for uri in uris]
ssd_input_batch = utils.prepare_tensor(inputs)

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [11]:
stats_dict, _ = ModelStatAnalyser.get_models_stats_dict(model_dict, input_batch, ssd_input_batch)

Analysing ssd
Analysing lenet
Analysing yolov5s
Analysing yolov5m
Analysing yolov5l
Analysing yolov5x
Analysing alexnet
Analysing vgg_11
Analysing vgg_13
Analysing vgg_16
Analysing vgg_19
Analysing vgg_11_bn
Analysing vgg_13_bn
Analysing vgg_16_bn
Analysing vgg_19_bn
Analysing resnet_18
Analysing resnet_34
Analysing resnet_50
Analysing resnet_101
Analysing resnet_152
Analysing squeezenet_1_0
Analysing squeezenet_1_1
Analysing densenet_121
Analysing densenet_169
Analysing densenet_201
Analysing densenet_161
Analysing inception_v3
Analysing googlenet
Analysing shufflenet_v2_x0_5
Analysing shufflenet_v2_x1_0
Analysing mobilenet_v2
Analysing mobilenet_v3_large
Analysing mobilenet_v3_small
Analysing resnext_50_32x4d
Analysing resnext_101_32x8d
Analysing wide_resnet_50_2
Analysing wide_resnet_101_2
Analysing mnasnet0_5
Analysing mnasnet1_0


In [19]:
ModelStatsAggregator.get_aggregate_kernel_stats_as_percentages(stats_dict)

{'7': 0.011862396204033215,
 '1': 0.5776986951364176,
 '3': 0.39541320680110714,
 '5': 0.015025701858442072}

In [20]:
ModelStatsAggregator.get_aggregate_stride_stats_per_kernel(stats_dict)

{'7': Counter({'2': 17, '1': 13}),
 '1': Counter({'1': 1431, '2': 28}),
 '3': Counter({'1': 910, '2': 90}),
 '11': Counter({'4': 1}),
 '5': Counter({'1': 28, '2': 10})}

In [22]:
ModelStatsAggregator.get_aggregate_in_channel_stats(stats_dict)

{'128': 392,
 '256': 334,
 '512': 261,
 '1024': 219,
 '192': 149,
 '64': 117,
 '160': 65,
 '96': 64,
 '48': 56,
 '320': 55,
 '384': 51,
 '24': 37,
 '768': 36,
 '3': 35,
 '640': 34,
 '32': 31,
 '576': 30,
 '116': 26,
 '16': 26,
 '2048': 25,
 '480': 25,
 '832': 21,
 '288': 21,
 '240': 20,
 '1280': 19,
 '960': 19,
 '72': 18,
 '80': 17,
 '1152': 14,
 '232': 14,
 '120': 14,
 '144': 14,
 '40': 13,
 '672': 13,
 '58': 11,
 '528': 10,
 '448': 8,
 '1536': 6,
 '224': 6,
 '352': 6,
 '416': 6,
 '864': 6,
 '896': 6,
 '928': 6,
 '992': 6,
 '1056': 6,
 '1248': 6,
 '704': 5,
 '736': 5,
 '800': 5,
 '1344': 5,
 '1440': 5,
 '1632': 5,
 '544': 4,
 '608': 4,
 '1088': 4,
 '1120': 4,
 '1184': 4,
 '1216': 4,
 '1728': 4,
 '184': 4,
 '112': 4,
 '12': 4,
 '1312': 3,
 '1376': 3,
 '1408': 3,
 '1472': 3,
 '1504': 3,
 '1568': 3,
 '1600': 3,
 '1824': 3,
 '1664': 2,
 '1696': 2,
 '1760': 2,
 '1792': 2,
 '336': 2,
 '432': 2,
 '624': 2,
 '720': 2,
 '1104': 2,
 '1200': 2,
 '1296': 2,
 '1392': 2,
 '1488': 2,
 '1584': 2,
 '1